<a href="https://colab.research.google.com/github/gopibhakta919/pyspark/blob/trainings/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 108 kB in 3s (41.7 kB/s)
Reading package lists... Done


In [20]:
from pyspark.sql.types import *
import pyspark.sql.functions as sqlfunc

myschema = StructType([
    StructField('ip_address' , StringType()),
    StructField('Country', StringType()),
    StructField('Domain Name', StringType()),
    StructField('Bytes_used', IntegerType())
])

In [21]:
df = spark.read.csv("challenge.csv", header = True, schema=myschema)

In [22]:
df1 = df.withColumn("Country_mexico" , sqlfunc.when(df.Country.isin('Mexico'), 'YES').otherwise('NO'))

In [23]:
df2 = df1.groupBy('Country_mexico').agg(sqlfunc.sum('Bytes_used').alias('Sum_bytes_used'))

In [25]:
df2.show()

+--------------+--------------+
|Country_mexico|Sum_bytes_used|
+--------------+--------------+
|           YES|          6293|
|            NO|        508076|
+--------------+--------------+



In [37]:
df3 = df1.groupBy('Country').agg(sqlfunc.countDistinct('ip_address').alias('ip_address_per_country'))
#df3 = df1.groupBy('Country').agg(sqlfunc.countDistinct(df1.ip_address).alias('ip_address_per_country'))

In [38]:
from pyspark.sql.functions import *
df3.sort(col("ip_address_per_country").desc()).show()

+--------------+----------------------+
|       Country|ip_address_per_country|
+--------------+----------------------+
|         China|                   172|
|     Indonesia|                   114|
|   Philippines|                    65|
|        Russia|                    56|
|        Brazil|                    35|
|        Poland|                    31|
|        Sweden|                    28|
|         Japan|                    25|
|      Portugal|                    23|
|Czech Republic|                    23|
|        France|                    21|
|          Peru|                    19|
|      Colombia|                    17|
| United States|                    15|
|       Ukraine|                    14|
|     Argentina|                    14|
|        Mexico|                    13|
|      Thailand|                    12|
|       Nigeria|                    11|
|        Canada|                    11|
+--------------+----------------------+
only showing top 20 rows

